# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("../Support/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
8,32.701514,0.468750,0.358132,0.019535,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978475,0.977538,0.974843,...,0.974133,0.003753,1,0.999102,0.999326,0.999102,0.999551,0.998877,0.999192,0.000229
10,17.883046,1.903112,0.239820,0.004907,None,150,"{'max_depth': None, 'n_estimators': 150}",0.979372,0.976640,0.973046,...,0.973954,0.004224,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
11,29.771885,1.837331,0.247698,0.013025,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976682,0.977538,0.975741,...,0.973954,0.003368,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
7,16.840396,0.394716,0.217171,0.006561,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978475,0.973046,0.973944,...,0.972696,0.003808,4,0.999326,0.999326,0.999326,0.999326,0.999102,0.999281,0.000090
4,13.763963,0.395511,0.192479,0.007927,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.975785,0.976640,0.970350,...,0.972337,0.003390,5,0.994609,0.993040,0.994612,0.993714,0.993714,0.993938,0.000602


In [4]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
11,29.374324,1.794224,0.236810,0.022923,None,300,"{'max_depth': None, 'n_estimators': 300}",0.979372,0.973046,0.974843,...,0.972876,0.004126,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
7,17.970798,0.673569,0.263890,0.042576,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977578,0.973046,0.973046,...,0.972696,0.002945,2,0.999102,0.998428,0.998877,0.998653,0.998877,0.998788,0.000229
10,17.768551,0.716947,0.227172,0.035438,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977578,0.973944,0.974843,...,0.972517,0.003806,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
8,33.679060,0.779486,0.370229,0.032133,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977578,0.972147,0.972147,...,0.971618,0.003499,4,0.998877,0.998653,0.999102,0.998877,0.998428,0.998788,0.000229
5,27.059792,0.970553,0.328191,0.040427,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.976682,0.969452,0.973944,...,0.970900,0.003809,5,0.993711,0.992591,0.993938,0.993714,0.992366,0.993264,0.000650
